In [ ]:
import os
import math
import h5py
import tensorflow as tf
import numpy as np

# Make data

In [ ]:
# data load
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# normalization
x_train = x_train.astype(np.float32)/255.
x_test = x_test.astype(np.float32)/255.

# flatten
x_train = x_train.reshape((-1, 28, 28, 1))
y_train = y_train.reshape((-1,  1))
x_test = x_test.reshape((-1, 28,28, 1))
y_test = y_test.reshape((-1, 1))
 
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(y_test.dtype)

In [ ]:
with h5py.File('myfile_train_x.hdf5', 'w') as f:
    group_x = f.create_group('x')
    
    print(group_x.name)
    print(list(f.keys()))

    group_x.create_dataset('image', data=x_train, dtype=np.float32)
    

In [ ]:
with h5py.File('myfile_train_y.hdf5', 'w') as f:
    
    group_y = f.create_group('y')
    print(group_y.name)
    print(list(f.keys()))
    
    group_y.create_dataset('label', data=y_train, dtype=np.uint8)

In [ ]:
with h5py.File('myfile_test_x.hdf5', 'w') as f:
    group_x = f.create_group('x')
    
    print(group_x.name)
    print(list(f.keys()))

    group_x.create_dataset('image', data=x_train, dtype=np.float32)

In [ ]:
with h5py.File('myfile_test_y.hdf5', 'w') as f:
    
    group_y = f.create_group('y')
    print(group_y.name)
    print(list(f.keys()))
    
    group_y.create_dataset('label', data=y_train, dtype=np.uint8)

# Load data

In [ ]:
class CIFAR10PyDataset(tf.keras.utils.Sequence):

    def __init__(self, path, keyword, batch_size, **kwargs):
        super().__init__(**kwargs)
        self.path = path
        self.keyword = keyword 
        self.batch_size = batch_size

        self.f = h5py.File(path, 'r')

        group_key = list(self.f.keys())[0]

        self.data = self.f[group_key][keyword]
        self.dsize = len(self.data)
        
        
        print('dsize:', self.dsize)
        
 

    def __del__(self):
        self.f.close()

    def __len__(self):
        # Return number of batches.
        return math.ceil(self.dsize / self.batch_size)

    def __getitem__(self, idx):
        # Return x, y for batch idx.
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, self.dsize)
        batch = self.data[low:high]

        return batch, batch

In [ ]:
data_train_x = CIFAR10PyDataset("./myfile_train_x.hdf5", "image", 128)
data_train_y = CIFAR10PyDataset("./myfile_train_y.hdf5", "label", 128)

# make model

In [ ]:
input_shape = (28,28,1)

In [ ]:
inputs = tf.keras.layers.Input(shape=input_shape, name='input')
layer1 = tf.keras.layers.Conv2D(4, (3,3), strides=1, activation="relu", padding='same', name="layer1")
layer2 = tf.keras.layers.Conv2D(16, (3,3), strides=2, activation="relu", padding='same', name="layer2")
layer3 = tf.keras.layers.Conv2D(32, (3,3), strides=2, activation="relu", padding='same', name="layer3") 
layer4 = tf.keras.layers.Conv2D(64, (3,3), strides=1, activation="relu", padding='same', name="layer4")
layer5 = tf.keras.layers.Conv2DTranspose(32, (3,3), strides=2, activation="relu", padding='same', name="layer5")
layer6 = tf.keras.layers.Conv2DTranspose(16, (3,3), strides=2, activation="relu", padding='same', name="layer6")
layer7 = tf.keras.layers.Conv2D(4, (3,3), strides=1, activation="relu", padding='same', name="layer7")
layer8 = tf.keras.layers.Conv2D(1, (3,3), strides=1, activation="relu", padding='same', name="layer8")

In [ ]:
x = layer1(inputs)
x = layer2(x)
x = layer3(x)
x = layer4(x)
x = layer5(x)
x = layer6(x)
x = layer7(x)
x = layer8(x)
model = tf.keras.models.Model(inputs, x, name="autoencoder")


In [ ]:
model.save('model_structure.h5', include_optimizer=False)
model.summary()

## compile model

In [ ]:
model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(), metrics=["mse"])

# Callbacks For Training

In [ ]:
def get_training_callbacks(names, base_path, model_name=None, dataloader=None, is_weights_only=False):
    callbacks=[]
    # callbacks =  tf.keras.callbacks.CallbackList()
    if 'ckeckpoint' in names:
        ckpt_dir = os.path.join(base_path, 'checkpoint', model_name )
        
        print(ckpt_dir)
        os.makedirs(ckpt_dir, exist_ok=True)

        if is_weights_only:
            ckeckpoint_dir = os.path.join(ckpt_dir, '{epoch:05d}_%s_{loss:.5e}.keras' % (model_name)) 
        else
            ckeckpoint_dir = os.path.join(ckpt_dir, '{epoch:05d}_%s_{loss:.5e}.weights.h5' % (model_name))
        callback_ckpt = tf.keras.callbacks.ModelCheckpoint(
                                filepath = ckeckpoint_dir,
                                monitor='val_loss',
                                verbose=1,
                                save_best_only=False,
                                save_weights_only=is_weights_only,
                                initial_value_threshold=initial_value_threshold )
        callbacks.append(callback_ckpt)
    if 'tensorboard' in names:
        tb_dir = os.path.join(base_path, 'board', model_name)
        os.makedirs(tb_dir, exist_ok=True)
        callback_tb = tf.keras.callbacks.TensorBoard( log_dir=tb_dir,
                                                       histogram_freq=10,
                                                       write_graph=True,
                                                       write_images=False)
        callbacks.append(callback_tb)

    return callbacks

In [ ]:
callbacks = get_training_callbacks(['ckeckpoint', 'tensorboard', ],
                                    base_path=".", model_name="autoencoder")

# Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
model_dir = os.path.join('board', 'autoencoder','train')
%tensorboard --logdir=model_dir

# Lets Train

In [ ]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():


model.fit(x=data_train_x, 
          epochs=100, 
          callbacks=callbacks)